In [1]:
conda install -c anaconda flask

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show flask

Name: Flask
Version: 1.1.2
Summary: A simple framework for building complex web applications.
Home-page: https://palletsprojects.com/p/flask/
Author: Armin Ronacher
Author-email: armin.ronacher@active-4.com
License: BSD-3-Clause
Location: c:\users\riyam\anaconda3\lib\site-packages
Requires: click, itsdangerous, Jinja2, Werkzeug
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
import pandas as pd
from flask import Flask, request, send_file
import os

app = Flask(__name__)

UPLOAD_FOLDER = "uploads"
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == "POST":
        file = request.files["file"]
        if file.filename == "":
            return "No file selected!"

        filepath = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(filepath)

        # Process the Excel file
        output_file = calculate_compound_interest(filepath)
        
        return send_file(output_file, as_attachment=True)

    return '''
    <!doctype html>
    <html>
    <body>
        <h2>Upload Excel File</h2>
        <form action="/" method="post" enctype="multipart/form-data">
            <input type="file" name="file">
            <input type="submit" value="Upload and Process">
        </form>
    </body>
    </html>
    '''

def calculate_compound_interest(filepath):
    df = pd.read_excel(filepath)

    expected_columns = {"Principal", "Rate", "StartDate", "MaturityDate", "CompoundsPerYear"}
    if not expected_columns.issubset(df.columns):
        return "Error: Excel must have Principal, Rate, StartDate, MaturityDate, and CompoundsPerYear columns."

    # Convert dates to datetime format
    df["StartDate"] = pd.to_datetime(df["StartDate"])
    df["MaturityDate"] = pd.to_datetime(df["MaturityDate"])

    # Calculate time in years (t)
    df["Time"] = (df["MaturityDate"] - df["StartDate"]).dt.days / 365.25

    # Apply compound interest formula
    df["Amount"] = df.apply(
        lambda row: row["Principal"] * (1 + row["Rate"] / row["CompoundsPerYear"]) ** (row["CompoundsPerYear"] * row["Time"]),
        axis=1
    )

    output_path = os.path.join(UPLOAD_FOLDER, "output_" + os.path.basename(filepath))
    df.to_excel(output_path, index=False)

    return output_path

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5001)  # Fixed restart issue & changed port


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Feb/2025 22:21:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2025 22:21:17] "GET /favicon.ico HTTP/1.1" 404 -


In [4]:
with open("app.py", "w") as f:
    f.write("""from datetime import datetime
import pandas as pd
from flask import Flask, request, send_file
import os

app = Flask(__name__)

UPLOAD_FOLDER = "uploads"
if not os.path.exists(UPLOAD_FOLDER):
 os.makedirs(UPLOAD_FOLDER)

@app.route("/", methods=["GET", "POST"])
def upload_file():
 if request.method == "POST":
     file = request.files["file"]
     if file.filename == "":
         return "No file selected!"

     filepath = os.path.join(UPLOAD_FOLDER, file.filename)
     file.save(filepath)

     # Process the Excel file
     output_file = calculate_compound_interest(filepath)
     
     return send_file(output_file, as_attachment=True)

 return '''
 <!doctype html>
 <html>
 <body>
     <h2>Upload Excel File</h2>
     <form action="/" method="post" enctype="multipart/form-data">
         <input type="file" name="file">
         <input type="submit" value="Upload and Process">
     </form>
 </body>
 </html>
 '''

def calculate_compound_interest(filepath):
 df = pd.read_excel(filepath)

 expected_columns = {"Principal", "Rate", "StartDate", "MaturityDate", "CompoundsPerYear"}
 if not expected_columns.issubset(df.columns):
     return "Error: Excel must have Principal, Rate, StartDate, MaturityDate, and CompoundsPerYear columns."

 # Convert dates to datetime format
 df["StartDate"] = pd.to_datetime(df["StartDate"])
 df["MaturityDate"] = pd.to_datetime(df["MaturityDate"])

 # Calculate time in years (t)
 df["Time"] = (df["MaturityDate"] - df["StartDate"]).dt.days / 365.25

 # Apply compound interest formula
 df["Amount"] = df.apply(
     lambda row: row["Principal"] * (1 + row["Rate"] / row["CompoundsPerYear"]) ** (row["CompoundsPerYear"] * row["Time"]),
     axis=1
 )

 output_path = os.path.join(UPLOAD_FOLDER, "output_" + os.path.basename(filepath))
 df.to_excel(output_path, index=False)

 return output_path

if __name__ == "__main__":
 app.run(debug=True, use_reloader=False, port=5001)""")
